In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
from zipfile import ZipFile

In [3]:
with ZipFile('drive/MyDrive/FinalDataset.zip','r') as zipObj:
  zipObj.extractall('/content')

In [4]:
# Discard the output of this cell.
#%%capture

# Install the required libraries.
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.4 MB/s eta 0:00:00
  Cloning https://github.com/TahaAnwar/pafy.git to /tmp/pip-install-i8j891z_/pafy_b12f77eb200d4dd89d01b2ff9c356b65
  Running command git clone --filter=blob:none --quiet https://github.com/TahaAnwar/pafy.git /tmp/pip-install-i8j891z_/pafy_b12f77eb200d4dd89d01b2ff9c356b65
  Resolved https://github.com/TahaAnwar/pafy.git to commit 2f3c473b3df7961721d07e1504675313afd1d2cb
  Preparing metadata (setup.py) ... done
  Created wheel for pafy: filename=pafy-0.5.5-py2.py3-none-any.whl size=35693 sha256=b026a8d2fdea1bc5ed9b6b7ac4d625afd7acea1cf5ea1907cd1ae37736a67125
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6xfhzco/wheels/38/75/72/035d5f91837fe6fdd9ee6ec71eadaf7829068a58f4789a63cc
Successfully built pafy


In [5]:
# Import the required libraries.
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [6]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [7]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

# Specify the directory containing the UCF50 dataset.
DATASET_DIR = "/content/FinalDataset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ["Abuse","Explosion","Fighting","RoadAccidents","Robbery","Shooting","Vandalism"]

In [8]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []

    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, frame = video_reader.read()

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return frames_list

In [9]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:

            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)

    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [10]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: Abuse
Extracting Data of Class: Explosion
Extracting Data of Class: Fighting
Extracting Data of Class: RoadAccidents
Extracting Data of Class: Robbery
Extracting Data of Class: Shooting
Extracting Data of Class: Vandalism


In [11]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [12]:
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state = seed_constant)

In [70]:
GlobalAveragePooling2D,
from keras.applications import VGG16
from keras.layers import GlobalAveragePooling2D
VGG16_base = VGG16(input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH,3),weights="imagenet",include_top=False)
# VGG16_output = VGG16_base.add(GlobalAveragePooling2D())
# Create a Sequential model
VGG16_model = Sequential()

# Add the VGG16 base model to the Sequential model
VGG16_model.add(VGG16_base)
VGG16_model.add(GlobalAveragePooling2D())


In [71]:
VGG16_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 global_average_pooling2d_8  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
VGG16_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 global_average_pooling2d_8  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
from keras.layers import TimeDistributed
video_input = Input(shape=(20, IMAGE_HEIGHT, IMAGE_WIDTH,3))
video_frames_encoded = TimeDistributed(VGG16_model)(video_input)

In [76]:
from keras.models import Model
from keras.layers import LSTM, Dropout, Dense
video_frames_encoded_sequence = LSTM(256)(video_frames_encoded)
video_frames_encoded_sequence = Dropout(0.5)(video_frames_encoded_sequence)
hidden_layer1 = Dense(1024, activation="relu")(video_frames_encoded_sequence)
hidden_layer1 = Dropout(0.5)(hidden_layer1)
hidden_layer2 = Dense(512, activation="relu")(hidden_layer1)
hidden_layer2 = Dropout(0.5)(hidden_layer2)
hidden_layer3 = Dense(256, activation="relu")(hidden_layer2)
hidden_layer3 = Dropout(0.25)(hidden_layer3)
hidden_layer4 =  Dense(128, activation="relu")(hidden_layer3)
hidden_layer4 = Dropout(0.25)(hidden_layer4)
hidden_layer5 =  Dense(64, activation="relu")(hidden_layer4)
hidden_layer5 = Dropout(0.25)(hidden_layer5)
outputs = Dense(7, activation="softmax")(hidden_layer5)
model2 = Model([video_input], outputs)

In [77]:
model2.compile(loss="categorical_crossentropy",
optimizer="adam",
metrics=["accuracy"])

In [78]:
history_model = model2.fit(x= features_train, y= labels_train, epochs=70, batch_size=4,shuffle= True, validation_split= 0.2)

Epoch 1/70
83/83 [==============================] - 21s 131ms/step - loss: 1.9857 - accuracy: 0.2371 - val_loss: 1.8466 - val_accuracy: 0.2771
Epoch 2/70
83/83 [==============================] - 8s 102ms/step - loss: 1.9400 - accuracy: 0.2310 - val_loss: 1.8559 - val_accuracy: 0.3012
Epoch 3/70
83/83 [==============================] - 9s 104ms/step - loss: 1.8955 - accuracy: 0.2492 - val_loss: 1.8429 - val_accuracy: 0.2771
Epoch 4/70
83/83 [==============================] - 9s 111ms/step - loss: 1.8699 - accuracy: 0.3100 - val_loss: 1.8741 - val_accuracy: 0.2771
Epoch 5/70
83/83 [==============================] - 9s 105ms/step - loss: 1.8929 - accuracy: 0.2584 - val_loss: 1.8588 - val_accuracy: 0.2771
Epoch 6/70
83/83 [==============================] - 9s 112ms/step - loss: 1.8799 - accuracy: 0.2644 - val_loss: 1.8502 - val_accuracy: 0.2771
Epoch 7/70
83/83 [==============================] - 9s 103ms/step - loss: 1.8478 - accuracy: 0.2614 - val_loss: 1.7824 - val_accuracy: 0.2771
Epoch

KeyboardInterrupt: ignored